In [1]:
import pandas as pd; import numpy as np; import requests; import re; from helium import *
import time, requests; from random import uniform; from bs4 import BeautifulSoup as bs

In [17]:
driver = start_chrome('https://www.espn.com/womens-college-basketball/standings/_/group/7')

In [21]:
go_to('https://www.espn.com/womens-college-basketball/standings/_/group/7')

In [248]:
# getting the final 2 numbers of the consecutive year
str(2023+1-2000)

'24'

In [249]:
# matching what the dropbox values will be
f'{2023}-{str(2023+1-2000)}'

'2023-24'

In [22]:
current_button = '2023-24'
for i in range(2023,2018, -1):
    year = f'{i}-{str(i+1-2000)}'
    click(Button(to_right_of=ComboBox('Big Ten')))
    select(current_button, year)
    click('Submit')
    time.sleep(15) # sleeping for 15 seconds before reading for 2 reasons: 
    #1. It prevents the website from realizing I'm using code to scrape the website rather than going through the website manually
    #2. It gives the website time to load in before the for loop copies the page source code to a txt file. 
    #time.sleep(5) # sleeping for five seconds because it keeps reading the standings for the 2020-21 season too quickly before it fully loads in
    # it was causing the 2021-22 and 2020-21 standings to be the same
    with open(f'espn_ncaaw_bigten_standings/{year}.txt', 'w', encoding = 'utf-8') as f:
        f.write(driver.page_source)
    current_button = year
    
kill_browser()

# Cleaning

In [2]:
# column names: Hardcoding column name "Team" in since it's only one object and the original 
# table on ESPN doesn't label the team column
standings = {}
for i in range(2023,2018, -1):
    col_names = ['Team']
    team_names = []
    cell_values = []
    year = f'{i}-{str(i+1-2000)}'
    with open(f'espn_ncaaw_bigten_standings/{year}.txt', encoding = 'utf-8') as f:
        soup = bs(f)
    # grabs team names
    for n in soup.find_all('tbody', class_ = "Table__TBODY")[0].find_all('tr'):
        name = n.find_all('span', class_ = 'hide-mobile')[0].get_text()
#        print(name)
        team_names.append(name)
    # grabs column names
    for c in soup.find_all('div', class_ = "Table__Scroller")[0].\
find_all('thead', class_ = 'Table__header-group Table__THEAD')[0].\
find_all('tr', class_ = 'Table__sub-header Table__TR Table__even')[0].\
find_all('th'):
        col_name = c.get_text()
       # print(col_name)
        col_names.append(col_name)
    # grabs data values
    for row in soup.find_all('div', class_ = "Table__Scroller")[0].\
find_all('tbody', class_ = 'Table__TBODY')[0].\
find_all('tr'):
        td = row.find_all('td')
        for val in range(0,len(td)):
            value = row.find_all('span')[val].get_text()
         #   print(value)
            cell_values.append(value)
    #print(len(cell_values))
    cells1 = np.array(pd.concat([pd.DataFrame(np.array(team_names)), pd.DataFrame(np.array(cell_values).\
                                                                         reshape(len(cell_values)//(len(col_names)-1), len(col_names)-1))], axis = 1))
    print(cells1)
    #saves data in dictionary
    standings[year] = pd.DataFrame(cells1, columns=col_names)
    #adds year to the dataset
    standings[year]['Year'] = year



[['Ohio State Buckeyes' '16-2' '-' '.889' '26-6' '.813' '16-2' '8-2' 'L1'
  '4-3' '3-3']
 ['Iowa Hawkeyes' '15-3' '1' '.833' '34-5' '.872' '15-1' '8-3' 'L1' '7-3'
  '7-3']
 ['Indiana Hoosiers' '15-3' '1' '.833' '26-6' '.813' '17-0' '7-4' 'L1'
  '4-4' '3-4']
 ['Michigan State Spartans' '12-6' '4' '.667' '22-9' '.710' '13-2' '6-4'
  'L2' '0-4' '0-4']
 ['Nebraska Cornhuskers' '11-7' '5' '.611' '23-12' '.657' '13-3' '5-7'
  'L1' '1-6' '1-6']
 ['Michigan Wolverines' '9-9' '7' '.500' '20-14' '.588' '11-4' '5-6' 'L2'
  '2-5' '2-5']
 ['Penn State Lady Lions' '9-9' '7' '.500' '22-13' '.629' '15-4' '5-6'
  'L1' '0-6' '0-6']
 ['Maryland Terrapins' '9-9' '7' '.500' '19-14' '.576' '12-3' '3-8' 'L2'
  '1-8' '1-8']
 ['Illinois Fighting Illini' '8-10' '8' '.444' '19-15' '.559' '12-5'
  '4-7' 'W5' '1-4' '1-5']
 ['Wisconsin Badgers' '6-12' '10' '.333' '15-17' '.469' '9-8' '5-7' 'L1'
  '0-4' '0-4']
 ['Minnesota Golden Gophers' '5-13' '11' '.278' '20-16' '.556' '15-6'
  '4-8' 'L1' '0-5' '0-5']
 ['Purdue B

In [3]:
for i in standings.keys():
    print(i)
    print(standings[i])

2023-24
                        Team   W-L  GB   PCT    W-L   PCT  HOME  AWAY STRK  \
0        Ohio State Buckeyes  16-2   -  .889   26-6  .813  16-2   8-2   L1   
1              Iowa Hawkeyes  15-3   1  .833   34-5  .872  15-1   8-3   L1   
2           Indiana Hoosiers  15-3   1  .833   26-6  .813  17-0   7-4   L1   
3    Michigan State Spartans  12-6   4  .667   22-9  .710  13-2   6-4   L2   
4       Nebraska Cornhuskers  11-7   5  .611  23-12  .657  13-3   5-7   L1   
5        Michigan Wolverines   9-9   7  .500  20-14  .588  11-4   5-6   L2   
6      Penn State Lady Lions   9-9   7  .500  22-13  .629  15-4   5-6   L1   
7         Maryland Terrapins   9-9   7  .500  19-14  .576  12-3   3-8   L2   
8   Illinois Fighting Illini  8-10   8  .444  19-15  .559  12-5   4-7   W5   
9          Wisconsin Badgers  6-12  10  .333  15-17  .469   9-8   5-7   L1   
10  Minnesota Golden Gophers  5-13  11  .278  20-16  .556  15-6   4-8   L1   
11       Purdue Boilermakers  5-13  11  .278  15-19  .44

In [8]:
#combining yearly standings into one dataframe
final_standings = pd.DataFrame()
for i in range(2023,2018,-1):
    year = f'{i}-{str(i+1-2000)}'
    final_standings = pd.concat([final_standings, standings[year]])
    final_standings.reset_index(drop = True, inplace= True)

In [9]:
final_standings

,Team,W-L,GB,PCT,W-L,PCT,HOME,AWAY,STRK,AP,USA,Year
0,Ohio State Buckeyes,16-2,-,.889,26-6,.813,16-2,8-2,L1,4-3,3-3,2023-24
1,Iowa Hawkeyes,15-3,1,.833,34-5,.872,15-1,8-3,L1,7-3,7-3,2023-24
2,Indiana Hoosiers,15-3,1,.833,26-6,.813,17-0,7-4,L1,4-4,3-4,2023-24
3,Michigan State Spartans,12-6,4,.667,22-9,.710,13-2,6-4,L2,0-4,0-4,2023-24
4,Nebraska Cornhuskers,11-7,5,.611,23-12,.657,13-3,5-7,L1,1-6,1-6,2023-24
...,...,...,...,...,...,...,...,...,...,...,...,...
65,Nebraska Cornhuskers,7-11,9,.389,17-13,.567,13-4,2-8,L2,1-5,0-5,2019-20
66,Minnesota Golden Gophers,5-13,11,.278,16-15,.516,9-8,6-6,L1,1-5,1-4,2019-20
67,Wisconsin Badgers,3-15,13,.167,12-19,.387,7-8,3-9,L1,0-6,0-6,2019-20
68,Illinois Fighting Illini,2-16,14,.111,11-19,.367,9-9,2-9,L7,0-4,0-4,2019-20


In [10]:
final_standings.to_csv('espn_ncaaw_bigten_standings/yearly_standings.csv')

# Coding for the 2023-24 Season before applying to For Loop

In [251]:
with open('espn_ncaaw_bigten_standings/2023-24.txt', encoding = 'utf-8') as f:
        soup = bs(f)

In [252]:
type(soup)

bs4.BeautifulSoup

In [253]:
name = soup.find_all('tbody', class_ = "Table__TBODY")[0].find_all('tr')[1].\
find_all('span', class_ = 'hide-mobile')[0].get_text()
name

'Iowa Hawkeyes'

In [254]:
#team names
teams = []
for i in soup.find_all('tbody', class_ = "Table__TBODY")[0].find_all('tr'):
    name = i.find_all('span', class_ = 'hide-mobile')[0].get_text()
    print(name)
    teams.append(name)

Ohio State Buckeyes
Iowa Hawkeyes
Indiana Hoosiers
Michigan State Spartans
Nebraska Cornhuskers
Michigan Wolverines
Penn State Lady Lions
Maryland Terrapins
Illinois Fighting Illini
Wisconsin Badgers
Minnesota Golden Gophers
Purdue Boilermakers
Northwestern Wildcats
Rutgers Scarlet Knights


In [256]:
soup.find_all('div', class_ = "Table__Scroller")[0].\
find_all('thead', class_ = 'Table__header-group Table__THEAD')[0].\
find_all('tr', class_ = 'Table__sub-header Table__TR Table__even')[0].\
find_all('th')[0].get_text()

'W-L'

In [257]:
# column names: Hardcoding column name "Team" in since it's only one object and the original 
# table on ESPN doesn't label the team column
cols = ['Team']
for i in soup.find_all('div', class_ = "Table__Scroller")[0].\
find_all('thead', class_ = 'Table__header-group Table__THEAD')[0].\
find_all('tr', class_ = 'Table__sub-header Table__TR Table__even')[0].\
find_all('th'):
    col_name = i.get_text()
    print(col_name)
    cols.append(col_name)

W-L
GB
PCT
W-L
PCT
HOME
AWAY
STRK
AP
USA


In [260]:
soup.find_all('div', class_ = "Table__Scroller")[0].\
find_all('tbody', class_ = 'Table__TBODY')[0].\
find_all('tr')[0].\
find_all('span')[0].get_text()

'16-2'

In [261]:
#getting data values
cells = []
for i in soup.find_all('div', class_ = "Table__Scroller")[0].\
find_all('tbody', class_ = 'Table__TBODY')[0].\
find_all('tr'):
    td = i.find_all('td')
    #using the length of td to go through each span tag and pull the value
    for j in range(0,len(td)):
        value = i.find_all('span')[j].get_text()
        print(value)
        cells.append(value)

16-2
-
.889
26-6
.813
16-2
8-2
L1
4-3
3-3
15-3
1
.833
34-5
.872
15-1
8-3
L1
7-3
7-3
15-3
1
.833
26-6
.813
17-0
7-4
L1
4-4
3-4
12-6
4
.667
22-9
.710
13-2
6-4
L2
0-4
0-4
11-7
5
.611
23-12
.657
13-3
5-7
L1
1-6
1-6
9-9
7
.500
20-14
.588
11-4
5-6
L2
2-5
2-5
9-9
7
.500
22-13
.629
15-4
5-6
L1
0-6
0-6
9-9
7
.500
19-14
.576
12-3
3-8
L2
1-8
1-8
8-10
8
.444
19-15
.559
12-5
4-7
W5
1-4
1-5
6-12
10
.333
15-17
.469
9-8
5-7
L1
0-4
0-4
5-13
11
.278
20-16
.556
15-6
4-8
L1
0-5
0-5
5-13
11
.278
15-19
.441
10-7
4-9
L1
0-6
0-6
4-14
12
.222
9-21
.300
6-9
3-9
L1
0-6
0-6
2-16
14
.111
8-24
.250
7-11
1-10
L5
0-4
0-4


In [262]:
print(teams)

['Ohio State Buckeyes', 'Iowa Hawkeyes', 'Indiana Hoosiers', 'Michigan State Spartans', 'Nebraska Cornhuskers', 'Michigan Wolverines', 'Penn State Lady Lions', 'Maryland Terrapins', 'Illinois Fighting Illini', 'Wisconsin Badgers', 'Minnesota Golden Gophers', 'Purdue Boilermakers', 'Northwestern Wildcats', 'Rutgers Scarlet Knights']


In [263]:
print(cols)

['Team', 'W-L', 'GB', 'PCT', 'W-L', 'PCT', 'HOME', 'AWAY', 'STRK', 'AP', 'USA']


In [264]:
print(cells)

['16-2', '-', '.889', '26-6', '.813', '16-2', '8-2', 'L1', '4-3', '3-3', '15-3', '1', '.833', '34-5', '.872', '15-1', '8-3', 'L1', '7-3', '7-3', '15-3', '1', '.833', '26-6', '.813', '17-0', '7-4', 'L1', '4-4', '3-4', '12-6', '4', '.667', '22-9', '.710', '13-2', '6-4', 'L2', '0-4', '0-4', '11-7', '5', '.611', '23-12', '.657', '13-3', '5-7', 'L1', '1-6', '1-6', '9-9', '7', '.500', '20-14', '.588', '11-4', '5-6', 'L2', '2-5', '2-5', '9-9', '7', '.500', '22-13', '.629', '15-4', '5-6', 'L1', '0-6', '0-6', '9-9', '7', '.500', '19-14', '.576', '12-3', '3-8', 'L2', '1-8', '1-8', '8-10', '8', '.444', '19-15', '.559', '12-5', '4-7', 'W5', '1-4', '1-5', '6-12', '10', '.333', '15-17', '.469', '9-8', '5-7', 'L1', '0-4', '0-4', '5-13', '11', '.278', '20-16', '.556', '15-6', '4-8', 'L1', '0-5', '0-5', '5-13', '11', '.278', '15-19', '.441', '10-7', '4-9', 'L1', '0-6', '0-6', '4-14', '12', '.222', '9-21', '.300', '6-9', '3-9', 'L1', '0-6', '0-6', '2-16', '14', '.111', '8-24', '.250', '7-11', '1-10', 'L

In [265]:
cells1 = teams + cells
cells1

['Ohio State Buckeyes',
 'Iowa Hawkeyes',
 'Indiana Hoosiers',
 'Michigan State Spartans',
 'Nebraska Cornhuskers',
 'Michigan Wolverines',
 'Penn State Lady Lions',
 'Maryland Terrapins',
 'Illinois Fighting Illini',
 'Wisconsin Badgers',
 'Minnesota Golden Gophers',
 'Purdue Boilermakers',
 'Northwestern Wildcats',
 'Rutgers Scarlet Knights',
 '16-2',
 '-',
 '.889',
 '26-6',
 '.813',
 '16-2',
 '8-2',
 'L1',
 '4-3',
 '3-3',
 '15-3',
 '1',
 '.833',
 '34-5',
 '.872',
 '15-1',
 '8-3',
 'L1',
 '7-3',
 '7-3',
 '15-3',
 '1',
 '.833',
 '26-6',
 '.813',
 '17-0',
 '7-4',
 'L1',
 '4-4',
 '3-4',
 '12-6',
 '4',
 '.667',
 '22-9',
 '.710',
 '13-2',
 '6-4',
 'L2',
 '0-4',
 '0-4',
 '11-7',
 '5',
 '.611',
 '23-12',
 '.657',
 '13-3',
 '5-7',
 'L1',
 '1-6',
 '1-6',
 '9-9',
 '7',
 '.500',
 '20-14',
 '.588',
 '11-4',
 '5-6',
 'L2',
 '2-5',
 '2-5',
 '9-9',
 '7',
 '.500',
 '22-13',
 '.629',
 '15-4',
 '5-6',
 'L1',
 '0-6',
 '0-6',
 '9-9',
 '7',
 '.500',
 '19-14',
 '.576',
 '12-3',
 '3-8',
 'L2',
 '1-8',
 '1-

In [266]:
#doesn't work as intended
np.array(cells1).reshape(len(cells1)//(len(cols)), len(cols))

array([['Ohio State Buckeyes', 'Iowa Hawkeyes', 'Indiana Hoosiers',
        'Michigan State Spartans', 'Nebraska Cornhuskers',
        'Michigan Wolverines', 'Penn State Lady Lions',
        'Maryland Terrapins', 'Illinois Fighting Illini',
        'Wisconsin Badgers', 'Minnesota Golden Gophers'],
       ['Purdue Boilermakers', 'Northwestern Wildcats',
        'Rutgers Scarlet Knights', '16-2', '-', '.889', '26-6', '.813',
        '16-2', '8-2', 'L1'],
       ['4-3', '3-3', '15-3', '1', '.833', '34-5', '.872', '15-1', '8-3',
        'L1', '7-3'],
       ['7-3', '15-3', '1', '.833', '26-6', '.813', '17-0', '7-4', 'L1',
        '4-4', '3-4'],
       ['12-6', '4', '.667', '22-9', '.710', '13-2', '6-4', 'L2', '0-4',
        '0-4', '11-7'],
       ['5', '.611', '23-12', '.657', '13-3', '5-7', 'L1', '1-6', '1-6',
        '9-9', '7'],
       ['.500', '20-14', '.588', '11-4', '5-6', 'L2', '2-5', '2-5',
        '9-9', '7', '.500'],
       ['22-13', '.629', '15-4', '5-6', 'L1', '0-6', '0-6', '9-

In [267]:
# trying with just reaaranging record cells then adding team names later
np.array(cells).reshape(len(cells)//(len(cols)-1), len(cols)-1)

array([['16-2', '-', '.889', '26-6', '.813', '16-2', '8-2', 'L1', '4-3',
        '3-3'],
       ['15-3', '1', '.833', '34-5', '.872', '15-1', '8-3', 'L1', '7-3',
        '7-3'],
       ['15-3', '1', '.833', '26-6', '.813', '17-0', '7-4', 'L1', '4-4',
        '3-4'],
       ['12-6', '4', '.667', '22-9', '.710', '13-2', '6-4', 'L2', '0-4',
        '0-4'],
       ['11-7', '5', '.611', '23-12', '.657', '13-3', '5-7', 'L1', '1-6',
        '1-6'],
       ['9-9', '7', '.500', '20-14', '.588', '11-4', '5-6', 'L2', '2-5',
        '2-5'],
       ['9-9', '7', '.500', '22-13', '.629', '15-4', '5-6', 'L1', '0-6',
        '0-6'],
       ['9-9', '7', '.500', '19-14', '.576', '12-3', '3-8', 'L2', '1-8',
        '1-8'],
       ['8-10', '8', '.444', '19-15', '.559', '12-5', '4-7', 'W5', '1-4',
        '1-5'],
       ['6-12', '10', '.333', '15-17', '.469', '9-8', '5-7', 'L1', '0-4',
        '0-4'],
       ['5-13', '11', '.278', '20-16', '.556', '15-6', '4-8', 'L1',
        '0-5', '0-5'],
       ['5-13', 

In [268]:
#combines team_names and stats and converts it back to an array
cells1 = np.array(pd.concat([pd.DataFrame(np.array(teams)), pd.DataFrame(np.array(cells).\
                                                                         reshape(len(cells)//(len(cols)-1), len(cols)-1))], axis = 1))
cells1

array([['Ohio State Buckeyes', '16-2', '-', '.889', '26-6', '.813',
        '16-2', '8-2', 'L1', '4-3', '3-3'],
       ['Iowa Hawkeyes', '15-3', '1', '.833', '34-5', '.872', '15-1',
        '8-3', 'L1', '7-3', '7-3'],
       ['Indiana Hoosiers', '15-3', '1', '.833', '26-6', '.813', '17-0',
        '7-4', 'L1', '4-4', '3-4'],
       ['Michigan State Spartans', '12-6', '4', '.667', '22-9', '.710',
        '13-2', '6-4', 'L2', '0-4', '0-4'],
       ['Nebraska Cornhuskers', '11-7', '5', '.611', '23-12', '.657',
        '13-3', '5-7', 'L1', '1-6', '1-6'],
       ['Michigan Wolverines', '9-9', '7', '.500', '20-14', '.588',
        '11-4', '5-6', 'L2', '2-5', '2-5'],
       ['Penn State Lady Lions', '9-9', '7', '.500', '22-13', '.629',
        '15-4', '5-6', 'L1', '0-6', '0-6'],
       ['Maryland Terrapins', '9-9', '7', '.500', '19-14', '.576',
        '12-3', '3-8', 'L2', '1-8', '1-8'],
       ['Illinois Fighting Illini', '8-10', '8', '.444', '19-15', '.559',
        '12-5', '4-7', 'W5', '1-

In [269]:
#final result
pd.DataFrame(cells1, columns=cols)

,Team,W-L,GB,PCT,W-L,PCT,HOME,AWAY,STRK,AP,USA
0,Ohio State Buckeyes,16-2,-,.889,26-6,.813,16-2,8-2,L1,4-3,3-3
1,Iowa Hawkeyes,15-3,1,.833,34-5,.872,15-1,8-3,L1,7-3,7-3
2,Indiana Hoosiers,15-3,1,.833,26-6,.813,17-0,7-4,L1,4-4,3-4
3,Michigan State Spartans,12-6,4,.667,22-9,.710,13-2,6-4,L2,0-4,0-4
4,Nebraska Cornhuskers,11-7,5,.611,23-12,.657,13-3,5-7,L1,1-6,1-6
5,Michigan Wolverines,9-9,7,.500,20-14,.588,11-4,5-6,L2,2-5,2-5
6,Penn State Lady Lions,9-9,7,.500,22-13,.629,15-4,5-6,L1,0-6,0-6
7,Maryland Terrapins,9-9,7,.500,19-14,.576,12-3,3-8,L2,1-8,1-8
8,Illinois Fighting Illini,8-10,8,.444,19-15,.559,12-5,4-7,W5,1-4,1-5
9,Wisconsin Badgers,6-12,10,.333,15-17,.469,9-8,5-7,L1,0-4,0-4
